In [1]:
# ===============================
# 3D nc 파일의 변수 목록 확인하는 코드
# ===============================


import netCDF4 as nc
import os

file_path = '/scratch/x3108a06/nc_data/OUT_3D_nc/rcemip_small_awing_96x96x74_1km_6s_300K_48_0000576000.nc'

def inspect_netcdf_variables(path):
    """
    지정된 경로의 NetCDF 파일에 포함된 변수들을 출력하는 함수입니다.

    Args:
        path (str): NetCDF 파일의 경로.
    """
    # 파일이 실제로 존재하는지 확인합니다.
    if not os.path.exists(path):
        print(f"오류: 파일을 찾을 수 없습니다. 경로를 확인해주세요: {path}")
        return

    try:
        # 'with' 구문을 사용하면 파일을 사용한 후 자동으로 닫아주어 편리합니다.
        # 파일을 읽기 모드('r')로 엽니다.
        with nc.Dataset(path, 'r') as dataset:
            
            # 파일 이름 출력
            print(f"--- 파일: {os.path.basename(path)} ---")

            # 파일에 변수가 있는지 확인합니다.
            if not dataset.variables:
                print("파일에 변수가 없습니다.")
                return
            
            print("\n[파일에 포함된 변수 목록]\n")
            
            # 보기 좋은 표 형태로 출력하기 위한 헤더
            print(f"{'변수 이름':<15} | {'차원 (Dimensions)':<30} | {'단위 (Units)'}")
            print("-" * 70)

            # dataset.variables는 딕셔너리처럼 파일 내의 모든 변수를 담고 있습니다.
            # 키(key)는 변수 이름입니다.
            for var_name in dataset.variables:
                variable = dataset.variables[var_name]
                
                # 변수의 차원 정보를 튜플 형태로 가져옵니다.
                dims = variable.dimensions
                
                # 변수의 단위 정보를 가져옵니다. 단위 정보가 없으면 'N/A'로 표시합니다.
                units = getattr(variable, 'units', 'N/A')
                
                # 변수 이름, 차원, 단위를 정해진 형식에 맞춰 출력합니다.
                print(f"{var_name:<15} | {str(dims):<30} | {units}")

    except Exception as e:
        print(f"파일을 읽는 중 오류가 발생했습니다: {e}")

# 함수를 실행하여 결과를 확인합니다.
inspect_netcdf_variables(file_path)


--- 파일: rcemip_small_awing_96x96x74_1km_6s_300K_48_0000576000.nc ---

[파일에 포함된 변수 목록]

변수 이름           | 차원 (Dimensions)                | 단위 (Units)
----------------------------------------------------------------------
x               | ('x',)                         | m
y               | ('y',)                         | m
z               | ('z',)                         | m
time            | ('time',)                      | d
p               | ('z',)                         | mb
U               | ('time', 'z', 'y', 'x')        | m/s       
V               | ('time', 'z', 'y', 'x')        | m/s       
W               | ('time', 'z', 'y', 'x')        | m/s       
PP              | ('time', 'z', 'y', 'x')        | Pa        
QRAD            | ('time', 'z', 'y', 'x')        | K/day     
TABS            | ('time', 'z', 'y', 'x')        | K         
QV              | ('time', 'z', 'y', 'x')        | g/kg      
QN              | ('time', 'z', 'y', 'x')        | g/kg      
QP              | 

In [2]:
import os
import xarray as xr
import numpy as np
import torch
from tqdm import tqdm

# ====== ⚙️ 1. 설정 (Settings) ⚙️ =======================================================
# --- 데이터 관련 설정 ---
days = 100                                  # 학습할 데이터 일수 (일)
gap_min = 5                                 # 데이터 간격 (분)
delta_min = 30                              # 몇 분 뒤를 예측할 것인지
start_idx = 576000                          # 시작 타임 인덱스 (40일 지점)

# --- 채널 관련 설정 ---
# 생성할 12개 채널의 이름 목록
CHANNEL_LIST = [
    # Single Level (4 channels)
    'tcwv', 't_lowest', 'u_lowest', 'v_lowest',
    # 850 hPa (4 channels)
    't850', 'z850', 'u850', 'v850',
    # 500 hPa (4 channels)
    't500', 'z500', 'u500', 'v500'
]
NUM_CHANNELS = len(CHANNEL_LIST)

# --- 경로 설정 ---
# 입력 데이터가 저장된 기본 경로
NC_2D_DIR = "/scratch/x3108a06/nc_data/OUT_2D_nc/"
NC_3D_DIR = "/scratch/x3108a06/nc_data/OUT_3D_nc/"

# 전처리된 .pt 파일을 저장할 디렉토리
var_name = '12ch'
base_dir = "/scratch/x3108a06/input_data/hybrid/"
input_dir = f'{base_dir}{var_name}_{gap_min}m'
pad_dir = f"{base_dir}{var_name}_d{delta_min}m_{gap_min}m"

# --- 물리 상수 ---
G_CONST = 9.80665  # 표준 중력가속도 (m/s^2)
# ======================================================================================


def get_12_channels_from_nc_files(timestep_index: int) -> np.ndarray:
    """
    주어진 타임스텝의 2D와 3D NetCDF 파일에서 12개 채널 데이터를 추출/계산하는 함수.
    - TCWV는 2D 파일의 PW 변수에서 가져옴.
    - 나머지 11개 채널은 3D 파일에서 계산함.

    Args:
        timestep_index (int): 처리할 파일의 타임스텝 인덱스.

    Returns:
        np.ndarray: (12, 96, 96) 모양의 Numpy 배열.
    """
    file_2d_path = os.path.join(NC_2D_DIR, f"rcemip_small_awing_96x96x74_1km_6s_300K_48_{timestep_index:010d}.2Dcom_1.nc")
    file_3d_path = os.path.join(NC_3D_DIR, f"rcemip_small_awing_96x96x74_1km_6s_300K_48_{timestep_index:010d}.nc")

    if not (os.path.exists(file_2d_path) and os.path.exists(file_3d_path)):
        return None

    with xr.open_dataset(file_2d_path) as ds_2d, xr.open_dataset(file_3d_path) as ds_3d:
        # --- 1. 단일 레벨 채널 계산 ---
        
        # 채널 1: 총가강수량 (Total Column Water Vapor)
        # 2D 파일의 'PW' 변수를 직접 사용
        tcwv = ds_2d['PW'].isel(time=0).values

        # 3D 파일에서 나머지 변수 불러오기
        z = ds_3d['z'].values
        p_hpa = ds_3d['p'].values
        pp = ds_3d['PP'].isel(time=0).values
        tabs = ds_3d['TABS'].isel(time=0).values
        u = ds_3d['U'].isel(time=0).values
        v = ds_3d['V'].isel(time=0).values

        # 채널 2: 최하층 기온 (Temperature at the lowest level) - 2m 기온 근사
        t_lowest = tabs[0, :, :]

        # 채널 3: 최하층 동서바람 (Eastward wind at the lowest level) - 10m 바람 근사
        u_lowest = u[0, :, :]

        # 채널 4: 최하층 남북바람 (Northward wind at the lowest level) - 10m 바람 근사
        v_lowest = v[0, :, :]

        # --- 2. 등압면 채널 계산 (Pressure Level Channels) ---
        geopotential = G_CONST * z
        full_pressure_pa = (p_hpa * 100)[:, np.newaxis, np.newaxis] + pp
        
        pressure_levels_hpa = [850.0, 500.0]
        interpolated_channels = []
        ny, nx = tcwv.shape

        for p_level_hpa in pressure_levels_hpa:
            target_pressure_pa = p_level_hpa * 100.0
            t_interp, z_interp, u_interp, v_interp = (np.zeros((ny, nx)) for _ in range(4))

            for j in range(ny):
                for i in range(nx):
                    p_profile = full_pressure_pa[:, j, i]
                    p_rev, t_rev, u_rev, v_rev, gp_rev = (
                        p_profile[::-1], tabs[:, j, i][::-1], u[:, j, i][::-1],
                        v[:, j, i][::-1], geopotential[::-1]
                    )
                    t_interp[j, i] = np.interp(target_pressure_pa, p_rev, t_rev)
                    z_interp[j, i] = np.interp(target_pressure_pa, p_rev, gp_rev)
                    u_interp[j, i] = np.interp(target_pressure_pa, p_rev, u_rev)
                    v_interp[j, i] = np.interp(target_pressure_pa, p_rev, v_rev)
            
            interpolated_channels.extend([t_interp, z_interp, u_interp, v_interp])

    # --- 3. 모든 채널을 하나로 합치기 ---
    all_channels = [tcwv, t_lowest, u_lowest, v_lowest] + interpolated_channels
    return np.stack(all_channels, axis=0)


def cyclic_pad_2d(x: torch.Tensor, pad: int) -> torch.Tensor:
    """주기적 경계 조건으로 2D 텐서를 패딩하는 함수."""
    x = torch.cat([x[..., -pad:, :], x, x[..., :pad, :]], dim=-2)
    x = torch.cat([x[..., :, -pad:], x, x[..., :, :pad]], dim=-1)
    return x


def main():
    """메인 실행 함수: 데이터 전처리 및 저장을 수행."""
    print("===== 12채널 데이터 전처리 시작 (하이브리드 방식) =====")
    os.makedirs(input_dir, exist_ok=True)
    
    # --- 1. 평균(mean) 및 표준편차(std) 계산 ---
    print("\n--- 1/2: 평균 및 표준편차 계산 중 ---")
    mean_array = np.zeros((NUM_CHANNELS,))
    mean2_array = np.zeros((NUM_CHANNELS,))
    count = 0
    
    loop_end_idx_train = start_idx + (days * 24 * 60 * 10) * 7 // 10
    for i in tqdm(range(start_idx, loop_end_idx_train + 1, gap_min * 100), desc="Calculating Mean/Std"):
        data_np = get_12_channels_from_nc_files(i)
        if data_np is None: continue

        mean_array += data_np.mean(axis=(1, 2))
        mean2_array += (data_np**2).mean(axis=(1, 2))
        count += 1
    
    if count == 0:
        print("오류: 평균/표준편차를 계산할 데이터를 찾지 못했습니다. 파일 경로와 인덱스를 확인하세요.")
        return

    mean = mean_array / count
    mean2 = mean2_array / count
    std = np.sqrt(mean2 - mean**2)

    mean_std_path = os.path.join(input_dir, f"mean_std_{days}d_{gap_min}m.npy")
    np.save(mean_std_path, np.array([mean, std]))
    print(f"평균/표준편차 저장 완료: {mean_std_path}")
    print("Mean:", mean)
    print("Std:", std)

    # --- 2. 정규화, 패딩 및 .pt 파일로 저장 ---
    print("\n--- 2/2: 데이터 정규화 및 저장 중 ---")
    loop_end_idx_full = start_idx + days * 24 * 60 * 10
    for i in tqdm(range(start_idx, loop_end_idx_full + 1, gap_min * 10), desc="Normalizing and Saving"):
        data_np = get_12_channels_from_nc_files(i)
        if data_np is None: continue
        
        data_tensor = torch.from_numpy(data_np).float()
        
        mean_tensor = torch.from_numpy(mean).float().view(NUM_CHANNELS, 1, 1)
        std_tensor = torch.from_numpy(std).float().view(NUM_CHANNELS, 1, 1)
        normalized_tensor = (data_tensor - mean_tensor) / std_tensor
        
        padded_tensor = cyclic_pad_2d(normalized_tensor, pad=16)

        torch.save(padded_tensor, os.path.join(input_dir, f"{i:010d}.pt"))

    print("\n===== 모든 전처리 작업 완료! =====")


if __name__ == '__main__':
    main()


===== 12채널 데이터 전처리 시작 (하이브리드 방식) =====

--- 1/2: 평균 및 표준편차 계산 중 ---


Calculating Mean/Std:  16%|█▌        | 325/2017 [03:36<23:00,  1.23it/s]

In [ ]:
# =======================
#      (x, y) stack
# =======================

# x, y를 첫번째 차원으로 쌓아서 (2, C, H, W) 모양의 텐서를 만드는 코드

os.makedirs(pad_dir, exist_ok=True)

# 파일 리스트 정렬
filelist = sorted([f for f in os.listdir(input_dir) if f.endswith('.pt')])
N = len(filelist)

# 몇번째 뒤와 x, y쌍으로 묶을 것인지
shift = delta_min // gap_min

for idx, fname in tqdm(enumerate(filelist)):  # type: int, str
    # 최종 결과 파일의 경로를 먼저 확인합니다.
    output_fpath = os.path.join(pad_dir, fname)

    # 만약 결과 파일이 이미 존재한다면, 이번 순서는 건너뜁니다.
    if os.path.exists(output_fpath):
        continue  # 다음 파일로 바로 넘어감

    fpath = os.path.join(input_dir, fname)
    data = torch.load(fpath, map_location='cpu')
    if not isinstance(data, torch.Tensor):
        print(f"{fname}: Not a tensor, skipped.")
        continue

    # 현재 파일
    # padded_0 = cyclic_pad_2d(data, 16)

    # shift만큼 뒤 파일(없으면 마지막 파일 사용)
    shifted_idx = min(idx + shift, N - 1)
    shifted_fname = filelist[shifted_idx]
    shifted_fpath = os.path.join(input_dir, shifted_fname)
    shifted_data = torch.load(shifted_fpath, map_location='cpu')
    
    # padded_1 = cyclic_pad_2d(shifted_data, 16)

    # (2, 128, 128) 저장
    out = torch.stack([data, shifted_data], dim=0)
    torch.save(out, os.path.join(pad_dir, fname))


In [6]:
# =======================
#    train test split
# =======================

import shutil
from math import floor


# 앞에서부터 70%를 train으로, 15%를 valid로, 15%를 test 셋으로 분리하여 저장하는 코드

def split_and_copy(src_dir, dst_base, prefix='', total_limit=None):
    os.makedirs(dst_base, exist_ok=True)
    files = sorted([f for f in os.listdir(src_dir) if os.path.isfile(os.path.join(src_dir, f))])
    if total_limit:
        files = files[:total_limit]
    n = len(files)
    n_train = floor(n * 0.7)
    n_valid = floor(n * 0.15)
    n_test = n - n_train - n_valid

    splits = [
        ('train', n_train),
        ('valid', n_valid),
        ('test', n_test),
    ]

    print(f"[{prefix}] Total: {n} -> train: {n_train}, valid: {n_valid}, test: {n_test}")

    idx = 0
    for split, count in splits:
        dst_dir = os.path.join(dst_base, prefix, split)
        os.makedirs(dst_dir, exist_ok=True)
        for i in tqdm(range(count), desc=split):
            src_path = os.path.join(src_dir, files[idx])
            dst_path = os.path.join(dst_dir, files[idx])
            idx += 1
            if os.path.exists(dst_path):
                continue  # 다음 파일로 바로 넘어감
            shutil.copy2(src_path, dst_path)


# ============== settings ====================
split_and_copy(
    src_dir=pad_dir,
    dst_base=base_dir,
    prefix=f'{var_name}_d{delta_min}m_{gap_min}m',
    total_limit=None
)
# ============================================

[PWUV_d30m_1m] Total: 144001 -> train: 100800, valid: 21600, test: 21601


test: 100%|██████████| 21601/21601 [00:00<00:00, 53621.80it/s]
